In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data/Restaurant_Reviews.csv")
data

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [3]:
import re
# nltk.download('stopwords')
from nltk.corpus import stopwords # Stop word like: the a an
from nltk.stem.porter import PorterStemmer # Root word like: loved->love

corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]',' ',data["Review"][i]) # Replace puntuation
    review = review.lower() # Make it lowercase
    review = review.split() # Split the word
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = " ".join(review)
    corpus.append(review)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer  # BagOfWords

cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = data["Liked"]

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [6]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
classifier.score(X_test,y_test)

0.73

In [7]:
from sklearn.ensemble import RandomForestClassifier

rd_classifier = RandomForestClassifier(n_estimators=100) # Defualt: n_estimators=100
rd_classifier.fit(X_train,y_train)
rd_classifier.score(X_test,y_test)

0.765

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train,y_train)
dt_classifier.score(X_test,y_test)

0.735

In [9]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()
log.fit(X_train,y_train)
log.score(X_test,y_test)

0.775

In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=6,activation="relu"))
model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"]) # Compile the model
model.fit(x=X_train,y=y_train,batch_size=50,epochs=100) # Train the model

2022-02-06 20:56:07.167833: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6937 - accuracy: 0.5113
Epoch 2/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6812 - accuracy: 0.6313
Epoch 3/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6679 - accuracy: 0.6988
Epoch 4/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6521 - accuracy: 0.7550
Epoch 5/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6315 - accuracy: 0.7912
Epoch 6/100
16/16 [==============================] - 0s 1ms/step - loss: 0.6077 - accuracy: 0.8200
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.5805 - accuracy: 0.8475
Epoch 8/100
16/16 [==============================] - 0s 1ms/step - loss: 0.5521 - accuracy: 0.8750
Epoch 9/100
16/16 [==============================] - 0s 1ms/step - loss: 0.5243 - accuracy: 0.9000
Epoch 10/100
16/16 [==============================] - 0s 1ms/step - loss: 0.4969 - accuracy: 0.9112
Epoch 11/

16/16 [==============================] - 0s 1ms/step - loss: 0.0520 - accuracy: 0.9887
Epoch 84/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0513 - accuracy: 0.9887
Epoch 85/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0506 - accuracy: 0.9887
Epoch 86/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0498 - accuracy: 0.9900
Epoch 87/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0491 - accuracy: 0.9900
Epoch 88/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0484 - accuracy: 0.9887
Epoch 89/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0477 - accuracy: 0.9887
Epoch 90/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0471 - accuracy: 0.9887
Epoch 91/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0464 - accuracy: 0.9887
Epoch 92/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0458 - accuracy: 0.9887
Epoch 93/100


In [11]:
from sklearn.metrics import accuracy_score

y_preds = model.predict(X_test)>0.5
accuracy_score(y_test,y_preds)

0.825

In [12]:
model.evaluate(X_test,y_test)  # Classification: Loss, Accuracy

7/7 [==============================] - 0s 942us/step - loss: 0.4896 - accuracy: 0.8250


[0.4895993769168854, 0.824999988079071]

In [43]:
sentence = "I hate this restaurant"

corpus = []
review = re.sub('[^a-zA-Z]',' ',sentence) # Replace puntuation
review = review.lower() # Make it lowercase
review = review.split() # Split the word
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
review = " ".join(review)
corpus.append(review)

In [44]:
corpus

['hate restaur']

In [45]:
test = cv.transform(corpus).toarray()
test

array([[0, 0, 0, ..., 0, 0, 0]])

In [46]:
model.predict(test)>0.5

array([[False]])